In [20]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Flatten
import pickle

In [21]:
IoT = pd.read_csv('../../data/merged_data.csv')
IoT = IoT.iloc[0:500]

IoT = IoT.dropna()

encoder=LabelEncoder()
IoT['fridge_temp_condition']=encoder.fit_transform(IoT['fridge_temp_condition'])
IoT['fridge_type']=encoder.fit_transform(IoT['fridge_type'])
IoT['door_state']=encoder.fit_transform(IoT['door_state'])
IoT['door_type']=encoder.fit_transform(IoT['door_type'])
IoT['ml_light_status_x']=encoder.fit_transform(IoT['ml_light_status_x'])
IoT['ml_type_x']=encoder.fit_transform(IoT['ml_type_x'])
IoT['gps_type']=encoder.fit_transform(IoT['gps_type'])
IoT['modbus_type']=encoder.fit_transform(IoT['modbus_type'])
IoT['ml_light_status_y']=encoder.fit_transform(IoT['ml_light_status_y'])
IoT['ml_type_y']=encoder.fit_transform(IoT['ml_type_y'])
IoT['thermostat_type']=encoder.fit_transform(IoT['thermostat_type'])
IoT['weather_type']=encoder.fit_transform(IoT['weather_type'])

IoT.head()

,date-time,fridge_temperature,fridge_temp_condition,fridge_label,fridge_type,door_state,door_sphone_signal,door_label,door_type,ml_motion_status_x,...,ml_type_y,thermostat_current_temperature,thermostat_status,thermostat_label,thermostat_type,weather_temperature,weather_pressure,weather_humidity,weather_label,weather_type
0,2019-03-31~12:36:52,13.10,0,0,0,0,False,0,0,1,...,0,25.0,0,0,0,31.788508,1.035,32.036579,0,0
1,2019-03-31~12:36:53,8.65,0,0,0,0,False,0,0,0,...,0,25.0,0,0,0,41.630997,1.035,30.886165,0,0
2,2019-03-31~12:36:54,2.00,1,0,0,0,False,0,0,0,...,0,25.0,0,0,0,42.256959,1.035,19.755908,0,0
3,2019-03-31~12:36:55,4.80,1,0,0,0,False,0,0,1,...,0,25.0,0,0,0,49.116581,1.035,78.949621,0,0
4,2019-03-31~12:36:56,10.70,0,0,0,0,False,0,0,1,...,0,25.0,0,0,0,24.017085,1.035,40.001059,0,0


In [22]:
x = IoT.drop(['door_label', 'date-time'], axis=1)
y = IoT['door_label']

features=len(x.columns)

x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)

In [23]:
from keras.wrappers.scikit_learn import KerasClassifier

def GRU_model():
    model2 = Sequential()
    
    model2.add(GRU(units = 128, activation='tanh', return_sequences=True, input_shape = (features,1)))
    model2.add(Dropout(0.2))
    model2.add(GRU(units = 64, activation='tanh', return_sequences=True))
    model2.add(Dropout(0.2))
    model2.add(Dense(1, activation='sigmoid')) 
    model2.add(Flatten())
    
    model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model2 = KerasClassifier(build_fn=GRU_model, epochs=10, batch_size=64)

/tmp/ipykernel_35491/780664580.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model2 = KerasClassifier(build_fn=GRU_model, epochs=10, batch_size=64)


In [24]:
# model2.save('../h5s/GRU.h5') 
filename = '../h5s/GRU.h5'
pickle.dump(model2, open(filename, 'wb'))

In [25]:
# model2.fit(x_train, np.asarray(y_train).astype('float32').reshape((-1,1)), epochs = 5, batch_size=32, validation_data=(x_test, np.asarray(y_test).astype('float32').reshape((-1,1))))

In [26]:
# predictions = model2.predict(x_test)
# y_pred = [round(x[0]) for x in predictions]
# y_pred

In [27]:
# for i in range(3):
# 	print(x_test.values[i].tolist(), '=>', y_pred[i], '( expected ', y_train.values[i], ')')

In [28]:
# score, acc = model2.evaluate(x_test, y_test, batch_size=1)

# print('Test score:', score)
# print('Test accuracy:', acc)